In [123]:
# read in data
df = pd.read_csv(write_data_dir / 'wmata_long_cleaned.csv')
df.Date = pd.to_datetime(df.Date)

/var/folders/g9/n5sfsl5n0z98mvb4hh40ftzr0000gn/T/ipykernel_81232/91401372.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'rank'] = df['Station'].map(rank_sort)
/var/folders/g9/n5sfsl5n0z98mvb4hh40ftzr0000gn/T/ipykernel_81232/91401372.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'color'] = '#d6a320'


In [ ]:
# grab the top 10 stations with highest total march ridership
top10_stations = df.groupby('Station')['Entries'].sum().nlargest(10).reset_index().Station
df = df[df.Station.isin(list(top10_stations))]
# create a ranking column so that both visualizations are sorted the same
rank_sort = dict(df.groupby('Station')['Entries'].sum())
df.loc[:, 'rank'] = df['Station'].map(rank_sort)
df.loc[:, 'color'] = '#d6a320'

In [153]:
# allow for a lot of rows
alt.data_transformers.enable('default', max_rows = None)

# create the selector field
selection = alt.selection_single(fields=['Station'],name='Random')
# create conditional color variables for the heat map
heatmap_color = alt.condition(selection,
                              alt.Color('Entries:Q',
                                scale=alt.Scale(scheme='orangered'),
                                        legend=alt.Legend(type='symbol',
                                                          orient='right',
                                                          direction = 'vertical',
                                                          legendX = .5,
                                                          legendY = 55
                                                         )
                        ),
                      alt.value('lightgray'))
# create conditional color variables for the bar chart
# have to create a dummy variable and adjust the color so that the legends line up
bar_color = alt.condition(selection,
                          alt.Color('color:N',
                                    scale=alt.Scale(domain=['#d6a320'], # use theme color
                                                    range=['#d6a320']),
                                    legend=None
                                   ),
                          alt.value('lightgray'))

# create the bar chart that shows total number of entries per station
bar=(alt.Chart(df)
 .mark_bar()
 .encode(x='sum(Entries):Q',
         y=alt.Y('Station:N',
         sort=alt.EncodingSortField(field='rank', op='mean', 
                            order='descending')),
         color=bar_color
        )
).add_selection(selection
).properties(
    title = "Total March Rides for the 10 Most Popular Metro Stations"
)

# create the heatmap that shows rides over time
heatmap = (alt.Chart(df)
 .mark_rect()
 .encode(x=alt.X('yearmonthdate(Date):O', title='WMATA Metro Entries'),
         y=alt.Y('Station:N', title='Metro Station',
                  sort=alt.EncodingSortField(field='rank',
                    order='descending')
                ),
         color = heatmap_color,
         tooltip=['Entries','Station:N']
        )
).properties(
    title = 'Average Wait Time in Hong Kong Hospitals'
)

heatmap.title ="Metro Rides Throughout March 2023"
heatmap.encoding.x.title = 'Date'
bar.encoding.x.title = 'Total Rides'
heatmap.encoding.y.title = 'Station'
alt.vconcat(bar, heatmap).resolve_scale(color='independent')

alt.VConcatChart(...)